In [7]:
import pandas as pd
import numpy as np
import torch, torch.nn
from transformers import BertModel, BertTokenizer
import re

import re, gc, os
import requests
from tqdm.auto import tqdm

In [8]:
def train_test_validation_splits(df):
    test,train_tot = df.loc[df['set']=='test'],df.loc[df['set']=='train']
    train, val = train_tot.loc[train_tot['validation']!=True], train_tot.loc[train_tot['validation']==True]
    return train,val, test

def embed_dataset(dataset_seqs, model, tokenizer, device, shift_left = 1, shift_right = -1):
  inputs_embedding = []

  for sample in tqdm(dataset_seqs):
    with torch.no_grad():
      ids = tokenizer.batch_encode_plus([sample], add_special_tokens=True, padding=True, is_split_into_words=True, return_tensors="pt")
      embedding = model(input_ids=ids['input_ids'].to(device))[0]
      inputs_embedding.append(embedding[0].detach().cpu().numpy()[shift_left:shift_right])

  return inputs_embedding

In [9]:
path_meltome = 'datasets/FLIP/splits/meltome/splits/mixed_split.csv'

splits_meltome = pd.read_csv(path_meltome, sep=',')
train, val, test = train_test_validation_splits(splits_meltome)
print('done')

done


In [10]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
model = model.eval()
if torch.cuda.is_available():
  model = model.half()

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
train_embeddings = embed_dataset(train.sequence, model, tokenizer, device)

 46%|████▋     | 10359/22335 [26:16<34:35,  5.77it/s] 

In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    
    def forward(self, input):
        return input.view(input.size(0), -1)   

class regressionHead(nn.Module):

    def __init__(self, channel_shape, input_shape, kernel):
        super(regressionHead, self).__init__()
        self.channel_dim = channel_shape
        self.input_shape = input_shape # pos0 = #channels, pos1 = #diagonal comps, or viseversa

        self.pooling = 

        self.FFNN = nn.Sequential(
            nn.BatchNorm1d(self.channel_dim),
            nn.Conv1d(self.channel_dim, self.channel_dim, kernel_size=kernel, stride=1, padding=kernel//2),
            nn.LeakyReLU(0.1),
            nn.Conv1d(self.channel_dim, self.channel_dim, kernel_size=kernel, stride=1, padding=kernel//2),
            nn.LeakyReLU(0.1),
            nn.Conv1d(self.channel_dim, self.channel_dim, kernel_size=kernel, stride=1, padding=kernel//2),
            nn.LeakyReLU(0.1),
            Flatten(),
            nn.Linear(self.channel_dim*self.input_shape[1], 1)
        )
        
        #self.encoder.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Conv1d):
            nn.init.kaiming_normal_(module.weight)

    def forward(self, x):
        z = self.FFNN(x)
        return z #z.permute(0,2,1) 

In [7]:



sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)
print(output)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0454,  0.1140, -0.0117,  ..., -0.0875, -0.1143,  0.0204],
         [ 0.0923,  0.1391, -0.0524,  ..., -0.1395, -0.0428,  0.0743],
         [ 0.1151,  0.0200, -0.0863,  ..., -0.0095, -0.1873,  0.1317],
         ...,
         [ 0.1079,  0.0977, -0.0583,  ..., -0.1277, -0.0649,  0.1289],
         [ 0.0546,  0.0364, -0.0782,  ..., -0.0302, -0.0602,  0.0890],
         [ 0.0515,  0.0571, -0.0693,  ..., -0.0394, -0.0663,  0.0977]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.2487,  0.2626, -0.2367,  ...,  0.2503,  0.2339, -0.2556]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)
